# 7.1 案例研究：Excel报表

用pandas的read_excel()函数读取一月的交易记录

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("sales_data/new/January.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9493 entries, 0 to 9492
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    9493 non-null   object        
 1   store             9493 non-null   object        
 2   status            9493 non-null   object        
 3   transaction_date  9493 non-null   datetime64[ns]
 4   plan              9493 non-null   object        
 5   contract_type     9493 non-null   object        
 6   amount            9493 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 519.3+ KB


# 7.2 使用pandas读写Excel文件

### 7.2.1 read_excel函数和ExcelFile类

sheet_name：要读取的工作表名称或索引。默认为0，即第一个工作表。<br>
skiprows：要跳过的行数或要跳过的行的列表。<br>
usecols：要读取的列。可以是列名的列表或列索引的列表。

In [3]:
df = pd.read_excel("xl/stores.xlsx", 
                  sheet_name="2019", 
                  skiprows=1, 
                  usecols="B:F")
df

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING
2,Chicago,4,Katelin,2020-01-31,NaN
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False
5,Las Vegas,11,Paul,2020-01-06,False


In [4]:
# 执行info()方法生成DataFrame的数据类型
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Store      6 non-null      object        
 1   Employees  6 non-null      int64         
 2   Manager    6 non-null      object        
 3   Since      5 non-null      datetime64[ns]
 4   Flagship   5 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 368.0+ bytes


Flagship的数据类型应该是bool而不是object：<br>
要修正，需要提供一个转换函数来处理某列中发生冲突的单元格<br>
【除了编写fix_missing函数，也可以提供一个lambda表达式】

In [5]:
def fix_missing(x):
    return False if x in ["", "MISSING"] else x

converters：用于转换数据的函数字典。

In [6]:
df = pd.read_excel("xl/stores.xlsx", 
                  skiprows=1, 
                  usecols="B:F",
                  converters={"Flagship": fix_missing})
df

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,False
2,Chicago,4,Katelin,2020-01-31,False
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False
5,Las Vegas,11,Paul,2020-01-06,False


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Store      6 non-null      object        
 1   Employees  6 non-null      int64         
 2   Manager    6 non-null      object        
 3   Since      5 non-null      datetime64[ns]
 4   Flagship   6 non-null      bool          
dtypes: bool(1), datetime64[ns](1), int64(1), object(2)
memory usage: 326.0+ bytes


read_excel()函数可以接受一个工作表名称列表，返回的是以DataFrame为值、工作表名称为键的一个字典

In [8]:
sheets = pd.read_excel("xl/stores.xlsx", 
                      sheet_name=["2019", "2020"], 
                      skiprows=1, 
                      usecols=["Store", "Employees"])
sheets["2019"].head(2)

,Store,Employees
0,New York,10
1,San Francisco,12


若源文件没有列标题，则设置参数header=None并通过names参数提供对应的列名<br>
【sheet_name也接受工作表切片】

skipfooter：指定要跳过的文件末尾的行数。

In [9]:
df = pd.read_excel("xl/stores.xlsx", 
                  sheet_name=0, 
                  skiprows=2, 
                  skipfooter=3, 
                  usecols="B:C,F", 
                  header=None, 
                  names=["Branch", "Employee_Count", "Is_Flagship"])
df

,Branch,Employee_Count,Is_Flagship
0,New York,10,False
1,San Francisco,12,MISSING
2,Chicago,4,NaN


为了处理NaN，可以把na_values和keep_default_na结合起来<br>
na_values：指定应该被视为缺失值的值<br>
keep_default_na：指定是否要将默认的缺失值（例如NaN）解析为NA

In [10]:
# 告诉pandas只将含有MISSING的单元格解释为NaN，除此之外的情况什么都不做
df = pd.read_excel("xl/stores.xlsx", 
                  sheet_name="2019", 
                  skiprows=1, 
                  usecols="B,C,F", 
                  skipfooter=2, 
                  na_values="MISSING", 
                  keep_default_na=False)
df

,Store,Employees,Flagship
0,New York,10,False
1,San Francisco,12,NaN
2,Chicago,4,
3,Boston,5,True


w：写入文件<br>
r：读取文件<br>
a：在文件末尾追加内容

In [11]:
f = open("output.txt", "w")
f.write("Some text")
f.close()

In [12]:
with open("output.txt", "a") as f:
    f.write("Hello World!")

nrows：要读取的行数

In [13]:
with pd.ExcelFile("xl/stores.xls") as f:
    df1 = pd.read_excel(f, "2019", skiprows=1, usecols="B:F", nrows=2)
    df2 = pd.read_excel(f, "2020", skiprows=1, usecols="B:F", nrows=2)
df1

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING


In [14]:
# 可以通过ExcelFile访问所有工作表的名称
stores = pd.ExcelFile("xl/stores.xlsx")
stores.sheet_names

['2019', '2020', '2019-2020']

In [15]:
# pandas通过URL读取Excel文件
url = "https://raw.githubusercontent.com/W2625/python-for-excel/master/xl/stores.xlsx"
pd.read_excel(url, skiprows=1, usecols="B:E", nrows=2)

,Store,Employees,Manager,Since
0,New York,10,Sarah,2018-07-20
1,San Francisco,12,Neriah,2019-11-02


### 7.2.2 to_excel()方法和ExcelWriter类

In [16]:
import numpy as np
import datetime as dt

In [17]:
data = [[dt.datetime(2020,1,1, 10, 13), 2.222, 1, True], 
       [dt.datetime(2020,1,2), np.nan, 2, False],
       [dt.datetime(2020,1,3), np.inf, 3, True]]
df = pd.DataFrame(data=data, 
                 columns=["Dates", "Floats", "Integers", "Booleans"])
df.index.name="index"
df

,Dates,Floats,Integers,Booleans
index,,,,
0,2020-01-01 10:13:00,2.222,1,True
1,2020-01-02 00:00:00,NaN,2,False
2,2020-01-03 00:00:00,inf,3,True


sheet_name：要写入的工作表名称<br>
startrow：被写入的第一行；startcol：被写入的第一列；两个的索引都是从0开始<br>
index和header：隐藏索引和/或标题，需要将参数设置为index=False和header=False<br>
na_rep和inf_rep：默认情况下，np.nan会被转换为空单元格，而NumPy用来表示无穷的np.inf会被转换为字符串inf。利用这两个参数可以修改默认行为<br>
freeze_panes：通过提供一个元组来冻结前几行和前几列，比如(2, 1)会冻结前两行和第一列

In [18]:
df.to_excel("xl/written_with_pandas.xlsx", 
            sheet_name="Output", 
            startrow=1, startcol=1, 
            index=True, header=True, 
            na_rep="<NA>", inf_rep="<INF>")

将多个DataFrame写入同一张或多张工作表，需要使用ExcelClass类

In [19]:
with pd.ExcelWriter("xl/writeen_with_pandas2.xlsx") as writer:
    df.to_excel(writer, sheet_name="Sheet1", startrow=1, startcol=1)
    df.to_excel(writer, sheet_name="Sheet1", startrow=10, startcol=1)
    df.to_excel(writer, sheet_name="Sheet2")